<a href="https://colab.research.google.com/github/Chunshan-Theta/mandarin_intent_classify/blob/main/examples/CBT_ABC%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 建置環境


In [1]:
!git clone --branch automation https://github.com/Chunshan-Theta/bert_mulit_label_chinese.git ./bert_mulit_label_chinese/

Cloning into './bert_mulit_label_chinese'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 55 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [2]:
!mv -v ./bert_mulit_label_chinese/* ./
!rm -rf ./bert_mulit_label_chinese/

renamed './bert_mulit_label_chinese/bert_wwm' -> './bert_wwm'
renamed './bert_mulit_label_chinese/data' -> './data'
renamed './bert_mulit_label_chinese/modeling.py' -> './modeling.py'
renamed './bert_mulit_label_chinese/mulit_label.py' -> './mulit_label.py'
renamed './bert_mulit_label_chinese/optimization.py' -> './optimization.py'
renamed './bert_mulit_label_chinese/output' -> './output'
renamed './bert_mulit_label_chinese/README.md' -> './README.md'
renamed './bert_mulit_label_chinese/requirements.txt' -> './requirements.txt'
renamed './bert_mulit_label_chinese/run_mulit.ipynb' -> './run_mulit.ipynb'
renamed './bert_mulit_label_chinese/tf_metrics.py' -> './tf_metrics.py'
renamed './bert_mulit_label_chinese/tokenization.py' -> './tokenization.py'


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 133kB 17.5MB/s 
     |████████████████████████████████| 184kB 42.9MB/s 
     |████████████████████████████████| 24.2MB 138kB/s 
     |████████████████████████████████| 122kB 50.0MB/s 
     |████████████████████████████████| 307kB 50.3MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 17.3MB 143kB/s 
     |████████████████████████████████| 1.0MB 55.6MB/s 
     |████████████████████████████████| 512kB 46.2MB/s 
     |████████████████████████████████| 645kB 49.3MB/s 
     |████████████████████████████████| 727kB 47.3MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 22.3MB 87kB/s 
     |████████████████████████████████| 25.9MB 84kB/s 
     |████████████████████████████████| 112kB 54.7MB/s 
     |████████████████████████████████| 3.8MB 50.7MB/s 

In [4]:
# 下載wwm模型
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":
    file_id = "1RoTQsXp2hkQ1gSRVylRIJfQxJUgkfJMW"
    destination = './bert_wwm/chinese_wwm_L-12_H-768_A-12.zip'
    download_file_from_google_drive(file_id, destination)

In [5]:
import os
import zipfile
file_path = f'./bert_wwm/chinese_wwm_L-12_H-768_A-12.zip'
out_path = "./bert_wwm/"
# zipfile example
def zip_list(file_path):
    zip = zipfile.ZipFile(file_path, 'r')
    print(zip.namelist())
    zip.extractall(out_path)

zip_list(file_path)

['publish/', 'publish/vocab.txt', 'publish/bert_model.ckpt.index', 'publish/bert_model.ckpt.data-00000-of-00001', 'publish/bert_config.json', 'publish/bert_model.ckpt.meta']


In [6]:
!mv -v ./bert_wwm/publish/* ./bert_wwm/
!rm -rf ./bert_wwm/publish/

renamed './bert_wwm/publish/bert_config.json' -> './bert_wwm/bert_config.json'
renamed './bert_wwm/publish/bert_model.ckpt.data-00000-of-00001' -> './bert_wwm/bert_model.ckpt.data-00000-of-00001'
renamed './bert_wwm/publish/bert_model.ckpt.index' -> './bert_wwm/bert_model.ckpt.index'
renamed './bert_wwm/publish/bert_model.ckpt.meta' -> './bert_wwm/bert_model.ckpt.meta'
renamed './bert_wwm/publish/vocab.txt' -> './bert_wwm/vocab.txt'


# 產生原始資料

In [7]:
# 下載wiki資料
import urllib.request
import progressbar
# https://dumps.wikimedia.org/zhwiki/latest/ 可以在這裡找到其他檔案
url = "https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-meta-current1.xml-p1p187712.bz2"
url = "https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles1.xml-p1p187712.bz2"
filename = "./data/zhwiki-latest-pages-meta-current1.xml.bz2"
pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

data = urllib.request.urlretrieve(url=url,filename=filename,reporthook=show_progress)

100% (200806036 of 200806036) |##########| Elapsed Time: 0:00:43 Time:  0:00:43


In [8]:
# 處理wiki文章
import logging
import sys
import warnings
warnings.filterwarnings(action ='ignore', category = UserWarning, module = 'gensim')
from gensim.corpora import WikiCorpus

# 將 wiki 資料集載下後進行 xml convert to txt 
class Wiki_to_txt(object):

    def __init__(self):
        # 用默認 Formatter 為日誌系統建立一個 StreamHandler ，設置基礎配置並加到 root logger 中
        logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

    # 使用方法 https://radimrehurek.com/gensim/corpora/wikicorpus.html
    def set_wiki_to_txt(self, wiki_data_path = None):
        if wiki_data_path == None:
            # 系統下參數
            if len(sys.argv) != 2:
                print("Please Usage: python3 " + sys.argv[0] + " wiki_data_path")
                exit()
            else:
                wiki_corpus = WikiCorpus(sys.argv[1], dictionary = {})
        else:
            wiki_corpus = WikiCorpus(wiki_data_path, dictionary = {})
        # wiki.xml convert to wiki.txt
        with open("./data/wiki_text.txt", 'w', encoding = 'utf-8') as output:
            text_count = 0
            
            for text in wiki_corpus.get_texts():
                # save use string(gensim)
                output.write(' '.join(text) + '\n')
                text_count += 1
                if text_count % 1000 == 0:
                    print(f"目前已處理 {text_count} 篇文章")
            print("轉檔完畢!")
            
if __name__ == "__main__":
    wiki_to_txt = Wiki_to_txt()
    # 將 wiki xml 轉換成 wiki txt
    wiki_to_txt.set_wiki_to_txt(filename)

目前已處理 1000 篇文章
目前已處理 2000 篇文章
目前已處理 3000 篇文章
目前已處理 4000 篇文章
目前已處理 5000 篇文章
目前已處理 6000 篇文章
目前已處理 7000 篇文章
目前已處理 8000 篇文章
目前已處理 9000 篇文章
目前已處理 10000 篇文章
目前已處理 11000 篇文章
目前已處理 12000 篇文章
目前已處理 13000 篇文章
目前已處理 14000 篇文章
目前已處理 15000 篇文章
目前已處理 16000 篇文章
目前已處理 17000 篇文章
目前已處理 18000 篇文章
目前已處理 19000 篇文章
目前已處理 20000 篇文章
目前已處理 21000 篇文章
目前已處理 22000 篇文章
目前已處理 23000 篇文章
目前已處理 24000 篇文章
目前已處理 25000 篇文章
目前已處理 26000 篇文章
目前已處理 27000 篇文章
目前已處理 28000 篇文章
目前已處理 29000 篇文章
目前已處理 30000 篇文章
目前已處理 31000 篇文章


2021-02-28 13:16:22,503 : INFO : finished iterating over Wikipedia corpus of 31626 documents with 11448587 positions (total 86064 articles, 11827400 positions before pruning articles shorter than 50 words)


轉檔完畢!


In [9]:
!pip install hanziconv
!pip install pypinyin

     |████████████████████████████████| 286kB 14.0MB/s 
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23215 sha256=c6de4a47fb25e2d18dc87fc8a4a929654a250897bf53b410f67614d3ddfcbb7f
  Stored in directory: /root/.cache/pip/wheels/03/d8/3c/c39898fa9c9ce6e34b0ab4c6604892462d440c743715c94054
Successfully built hanziconv
     |████████████████████████████████| 1.3MB 17.9MB/s 


In [10]:
# -*- coding: utf-8 -*-
import jieba
import logging
from hanziconv import HanziConv
import urllib.request
from pypinyin import lazy_pinyin,Style


ENV_simplified_to_traditional = True
ENV_lazy_pinyin = False
ENV_lazy_pinyin_style=Style.TONE3


# 進行斷詞並過濾 stopword
class Segmentation(object):

    def __init__(self):
        # 用默認 Formatter 為日誌系統建立一個 StreamHandler ，設置基礎配置並加到 root logger 中
        logging.basicConfig(format = "%(asctime)s : %(levelname)s : %(message)s", level = logging.INFO)
        self.stopwordset = set()
        
    # 讀取 stopword 辭典，並存到 stopwordset
    def set_stopword(self):
        stopwords_url ="https://raw.githubusercontent.com/Chunshan-Theta/mandarin_fields_intent/main/stopword.txt" # it's a file like object and works just like a file
        for line in urllib.request.urlopen(stopwords_url):
            self.stopwordset.add(line.decode("utf-8").strip("\n"))
        #print(self.stopwordset)
        print("StopWord Set 已儲存!")

    # 簡 to 繁
    def simplified_to_traditional(self):
        logging.info("等待中..(簡 to 繁)")
        traditional = open("./data/traditional.txt", "w", encoding = "utf-8")
        with open("./data/wiki_text.txt", "r", encoding = "utf-8") as simplified:
            simplifiedData = list(simplified)
            idx = 0
            for s in simplifiedData:
                if ENV_simplified_to_traditional:
                    traditional.write(HanziConv.toTraditional(s))
                else:
                    traditional.write(s)
                idx +=1
                if idx % 1000 == 0:
                    print(f"{idx} /{len(simplifiedData)}篇文章已成功簡體轉繁體")
        print("成功簡體轉繁體!")
        traditional.close()

    # 斷詞(Segmentation)並過濾掉停用詞(Stop Word) 
    def segmentation(self):
        logging.info("等待中..(jieba 斷詞，並過濾停用詞)")
        segmentation = open("./data/segmentation.txt", "w", encoding = "utf-8")
        with open("./data/traditional.txt", "r", encoding = "utf-8") as Corpus:
            CorpusData = list(Corpus)
            for idx, sentence in enumerate(CorpusData):
                sentence = sentence.strip("\n")
                pos = jieba.cut(sentence, cut_all = False)
                for term in pos:
                    if term not in self.stopwordset:
                        if ENV_lazy_pinyin:
                            term_pinyin = "".join(lazy_pinyin(term,style=ENV_lazy_pinyin_style))
                            segmentation.write(term_pinyin + " ")
                        else:
                            segmentation.write(term + " ")
                segmentation.write("\n")
                if idx % 100 == 0:
                    print(f"{idx}/ {len(CorpusData)}篇文章已 jieba 斷詞完畢，並完成過濾停用詞!")
        print("全部篇文章已jieba 斷詞完畢，並已完成過濾停用詞!")
        segmentation.close()

    # 斷詞(Segmentation)並過濾掉停用詞(Stop Word)
    @staticmethod
    def CorpusSegmentation(Corpus,stopwordset):
        formattedCorpus = []
        for sentence in Corpus:
            if ENV_simplified_to_traditional:
                sentence = HanziConv.toTraditional(sentence.strip("\n"))
            else:
                sentence = sentence.strip("\n")
            pos = jieba.cut(sentence, cut_all = False)
            sentence = ""
            for term in pos:
                if term not in stopwordset:
                    if ENV_lazy_pinyin:
                        term_pinyin = "".join(lazy_pinyin(term,style=ENV_lazy_pinyin_style))
                        sentence += (term_pinyin + " ")
                    else:
                        sentence += (term + " ")
            formattedCorpus.append(sentence)
        return formattedCorpus

if __name__ == "__main__":
    segmentation = Segmentation()
    # 讀取停用詞辭典
    segmentation.set_stopword()
    # data 進行簡體轉繁體
    segmentation.simplified_to_traditional()
    # 進行 jieba 斷詞同步過濾停用詞，並產生辭典
    #segmentation.segmentation()

2021-02-28 13:16:30,786 : INFO : 等待中..(簡 to 繁)


StopWord Set 已儲存!
1000 /31626篇文章已成功簡體轉繁體
2000 /31626篇文章已成功簡體轉繁體
3000 /31626篇文章已成功簡體轉繁體
4000 /31626篇文章已成功簡體轉繁體
5000 /31626篇文章已成功簡體轉繁體
6000 /31626篇文章已成功簡體轉繁體
7000 /31626篇文章已成功簡體轉繁體
8000 /31626篇文章已成功簡體轉繁體
9000 /31626篇文章已成功簡體轉繁體
10000 /31626篇文章已成功簡體轉繁體
11000 /31626篇文章已成功簡體轉繁體
12000 /31626篇文章已成功簡體轉繁體
13000 /31626篇文章已成功簡體轉繁體
14000 /31626篇文章已成功簡體轉繁體
15000 /31626篇文章已成功簡體轉繁體
16000 /31626篇文章已成功簡體轉繁體
17000 /31626篇文章已成功簡體轉繁體
18000 /31626篇文章已成功簡體轉繁體
19000 /31626篇文章已成功簡體轉繁體
20000 /31626篇文章已成功簡體轉繁體
21000 /31626篇文章已成功簡體轉繁體
22000 /31626篇文章已成功簡體轉繁體
23000 /31626篇文章已成功簡體轉繁體
24000 /31626篇文章已成功簡體轉繁體
25000 /31626篇文章已成功簡體轉繁體
26000 /31626篇文章已成功簡體轉繁體
27000 /31626篇文章已成功簡體轉繁體
28000 /31626篇文章已成功簡體轉繁體
29000 /31626篇文章已成功簡體轉繁體
30000 /31626篇文章已成功簡體轉繁體
31000 /31626篇文章已成功簡體轉繁體
成功簡體轉繁體!


# 產生訓練集

In [19]:
import re
import os


def check_match_re(rule, string):
    rule = rule.replace("｜", "|")
    regex = re.compile(rule)
    if regex.match(string) is None:
        return None
    else:
        return regex.match(string).string

# types = {
#     "引發事件": [
#         r".*(劈腿|分手|離婚)+.*"
#     ],
#     "認知想法": [
#         r".*(他|她|你|妳)+.*(認為|覺得|不想|想要)+.*"
#     ],
#     "感受內在感受": [
#         r"[^他她你妳們它祂]*?(覺得|感覺|感到|發覺)+.*(尊重|孤單|難過|生氣|自卑|寂寞|開心|愉悅|痛苦|苦楚|開心|心急|心煩|害怕|恐懼|擔心|嫉妒|畏懼|氣憤|羞|非常|極度)+.*",
#         r".*(孤單|難過|生氣|自卑|寂寞|開心|愉悅|痛苦|苦楚|開心|心急|心煩|害怕|恐懼|擔心|嫉妒|畏懼|氣憤|羞)+.*",
#     ]
# }


types = {
    "感受內在感受_孤單小孩": [
        r".*(寂寞|孤單)+.*"
    ],
    "感受內在感受_憤怒小孩": [
        r".*(生氣|憤怒|不原諒)+.*"
    ],
    "感受內在感受_創傷小孩": [
        r".*(害怕|恐懼)+.*",
        r".*(缺乏|沒有|不)+.*(自信|信心)+.*"
    ],
    "認知想法_控制欲": [
        r".*(應該|一定)+(會|要|很)+.*"
    ],
    "認知想法_差評": [
        r".*(他|她|你|妳)+.*(非常|超級|真)+.*(糟糕|差勁|無法理解)+.*"
    ],
    "事件_感情": [
        r".*(分手|離婚|外遇)+.*"
    ],
    "事件_金錢": [
        r".*(債務|經濟壓力|討債|欠錢)+.*"
    ],
    "事件_成長": [
        r".*(失敗|考很差|討債|欠錢)+.*",
        r".*(考試|分數|考績|考試)+(超級|非常|很)?(差|不好|失敗)+.*"
    ],
}

if not os.path.isdir("./data/data-wiki/") :
    os.makedirs("./data/data-wiki/")
sentences_classify = {}
with open("./data/traditional.txt", "r", encoding="utf-8") as Corpus, open("./data/data-wiki/train.tsv", "w", encoding="utf-8") as f,  open("./data/data-wiki/test.tsv", "w", encoding="utf-8") as o:
    CorpusData = list(Corpus)
    for t, ks in types.items():
        for k in ks:
            print(f"{t}\t{k}")
            for idx, sentence in enumerate(CorpusData):
                sentences = sentence.split(" ")
                for s in sentences:
                    s = check_match_re(rule=k, string=s)
                    if s is not None and len(s) > 12:
                        if s not in sentences_classify:
                            sentences_classify[s] = []
                        if t not in sentences_classify[s]:
                            sentences_classify[s].append(t)
    idx = 0
    for s, ts in sentences_classify.items():
        idx +=1
        t = str(ts).replace("\'", "")
        if idx %10 != 0:
          f.write(f"{t}\t{s}\n")
        else:
          o.write(f"{t}\t{s}\n")

感受內在感受_孤單小孩	.*(寂寞|孤單)+.*
感受內在感受_憤怒小孩	.*(生氣|憤怒|不原諒)+.*
感受內在感受_創傷小孩	.*(害怕|恐懼)+.*
感受內在感受_創傷小孩	.*(缺乏|沒有|不)+.*(自信|信心)+.*
認知想法_控制欲	.*(應該|一定)+(會|要|很)+.*
認知想法_差評	.*(他|她|你|妳)+.*(非常|超級|真)+.*(糟糕|差勁|無法理解)+.*
事件_感情	.*(分手|離婚|外遇)+.*
事件_金錢	.*(債務|經濟壓力|討債|欠錢)+.*
事件_成長	.*(失敗|考很差|討債|欠錢)+.*
事件_成長	.*(考試|分數|考績|考試)+(超級|非常|很)?(差|不好|失敗)+.*


# 訓練

In [20]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""BERT finetuning runner."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import csv
import os
import modeling
import optimization
import tokenization
import tensorflow as tf
import pickle
import json

from sklearn.utils import class_weight
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import metrics_impl
from tensorflow.python.ops import variable_scope
from tensorflow.python.distribute import distribution_strategy_context

import tf_metrics

model_tag = "3"
data_dir = f'data/data-wiki/'
bert_config_file = 'bert_wwm/bert_config.json'
task_name = "customized"
output_dir = f'./output-wiki-{model_tag}/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
init_checkpoint = 'bert_wwm/bert_model.ckpt'
do_lower_case = True
max_seq_length = 20
use_tpu = False
use_one_hot_embeddings = False
do_train = True
do_eval = True
do_predict = True
train_batch_size = 32
eval_batch_size = 32
predict_batch_size = 32
learning_rate = 5e-5
num_train_epochs = 25
warmup_proportion = 0.1
save_checkpoints_steps = 100
iterations_per_loop = 1000
vocab_file = './bert_wwm/vocab.txt'
master = None
num_tpu_cores = 8
crf_loss_method=True
do_export = False
export_dir = None
dataset = ["train.tsv", "train.tsv", "test.tsv"]


####
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        """Constructs a InputExample.
        Args:
          guid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text = text
        self.label = label


class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 input_ids,
                 input_mask,
                 segment_ids,
                 label_ids,
                 is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        self.is_real_example = is_real_example


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for prediction."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with tf.gfile.Open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if len(line) > 0:
                    lines.append(line)

            return lines

    @classmethod
    def _read_data(cls, input_file):
        """Reads a BIO data."""
        with open(input_file) as f:
            lines = []
            words = []
            labels = []
            for line in f:
                contends = line.strip()
                word = line.strip().split(' ')[0]
                label = line.strip().split(' ')[-1]
                if contends.startswith("-DOCSTART-"):
                    words.append('')
                    continue
                # if len(contends) == 0 and words[-1] == '。':
                if len(contends) == 0:
                    l = ' '.join([label for label in labels if len(label) > 0])
                    w = ' '.join([word for word in words if len(word) > 0])
                    lines.append([l, w])
                    words = []
                    labels = []
                    continue
                words.append(word)
                labels.append(label)
            return lines


class customizedProcessor(DataProcessor):

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, dataset[0])), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, dataset[1])), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, dataset[2])), "test")

    def get_labels_info(self):
        labels = []
        label_map = {}
        label_map_file = os.path.join( output_dir, "label_map.txt")
        lines = self._read_tsv(os.path.join( data_dir, dataset[0]))

        for line in lines:
            if len(line) == 0:
                continue
            try:
                line_of_labels = line[0].strip("[]").split(', ')
            except IndexError as e:
                print(f"IndexError:{line}")
                raise e
            for label in line_of_labels:
                labels.append(label)



        all_labels = labels  # for cal intent_weights

        labels = sorted(set(labels), reverse=False)
        num_labels = sorted(set(labels), reverse=True).__len__()

        intent_class_weights = class_weight.compute_class_weight('balanced',
                                                                 labels,
                                                                 all_labels)


        with tf.gfile.GFile(label_map_file, "w") as writer:
            for (i, label) in enumerate(labels):
                label_map[label] = i
                writer.write("{}:{}\n".format(i, label))



        return label_map, num_labels, intent_class_weights

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            try:
                label = tokenization.convert_to_unicode(line[0])
                text = tokenization.convert_to_unicode(line[1])
                examples.append(InputExample(guid=guid, text=text, label=label))
            except IndexError as e:
                print(line)
                raise e
        return examples


def write_tokens(tokens, mode):
    if mode == "test":
        path = os.path.join( output_dir, "token_" + mode + ".txt")
        wf = open(path, 'a')
        for token in tokens:
            if token != "**NULL**":
                wf.write(token + '\n')
        wf.close()

def convert_single_example(ex_index, example, label_map,
                           max_seq_length, tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=[0] * len(label_map),
            is_real_example=False)

    tokens_list = example.text
    tokens = []
    tags = []

    tokens.append("[CLS]")

    for i, word in enumerate(tokens_list):
        token = tokenizer.tokenize(word)
        tokens.extend(token)

    tokens.append("[SEP]")


    if len(tokens) >= max_seq_length:
        tokens = tokens[0:max_seq_length]

    

    segment_ids = [0] * max_seq_length
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)
    label_list = example.label.strip('[]').split(', ')
    multi_label_list = [0] * len(label_map)

    for label in label_list:
        label_index = label_map[label]
        multi_label_list[label_index] = 1

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if ex_index < 5:
        tf.logging.info("*** Example ***")
        tf.logging.info("guid: %s" % (example.guid))
        tf.logging.info("label: %s" % (example.label))
        tf.logging.info("tokens: %s" % " ".join(
            [tokenization.printable_text(x) for x in tokens]))
        tf.logging.info(
            "input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info(
            "input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info(
            "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        #tf.logging.info("tags_ids: %s" % " ".join([str(x) for x in tags_ids]))
    feature = InputFeatures(
        label_ids=multi_label_list,
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    return feature


def file_based_convert_examples_to_features(
        examples, label_map, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""
    writer = tf.python_io.TFRecordWriter(output_file)
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            tf.logging.info(
                "Writing example %d of %d" % (ex_index, len(examples)))
        feature = convert_single_example(ex_index, example, label_map,
                                        max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(
                int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["label_ids"] = create_int_feature(feature.label_ids)
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])

        tf_example = tf.train.Example(
            features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder, num_labels):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "label_ids": tf.FixedLenFeature([num_labels], tf.int64),
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 label_ids,  intent_num_labels, intent_class_weights,
                 use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    intent_output_layer = model.get_pooled_output()
    sequence_output_layer = model.get_sequence_output()

    hidden_size = sequence_output_layer.shape[-1].value

    ## intent loss
    intent_output_weights = tf.get_variable(
        "intent_output_weights", [intent_num_labels, hidden_size],
        # initializer=tf.truncated_normal_initializer(stddev=0.02)
        initializer=tf.contrib.layers.xavier_initializer()
    )

    intent_output_bias = tf.get_variable(
        "intent_output_bias", [intent_num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("intent_loss"):
        if is_training:
            # I.e., 0.1 dropout
            intent_output_layer = tf.nn.dropout(intent_output_layer, keep_prob=0.9)

        intent_logits = tf.matmul(intent_output_layer, intent_output_weights, transpose_b=True)
        intent_logits = tf.nn.bias_add(intent_logits, intent_output_bias, name='intent_logits')
        intent_probabilities = tf.nn.sigmoid(intent_logits, name='intent_probabilities')
        # classes_weights = tf.constant([1] * intent_num_labels, dtype=tf.float32)

        classes_weights = tf.constant(intent_class_weights, dtype=tf.float32)

        weighted_cross_entropy = tf.nn.weighted_cross_entropy_with_logits(labels=tf.cast(label_ids, tf.float32),
                                                                          logits=intent_logits,
                                                                          pos_weight=classes_weights)
        intent_per_example_loss = tf.reduce_sum(weighted_cross_entropy, axis=-1)



    loss = tf.reduce_mean(intent_per_example_loss)

    return (loss, -1, intent_logits, -1, intent_probabilities)


def f1_score(labels, predictions, weights=None, num_thresholds=200,
             metrics_collections=None, updates_collections=None, name=None):
    with variable_scope.variable_scope(
            name, 'f1', (labels, predictions, weights)):
        predictions, labels, weights = metrics_impl._remove_squeezable_dimensions(  # pylint: disable=protected-access
            predictions=predictions, labels=labels, weights=weights)
        # To account for floating point imprecisions / avoid division by zero.
        epsilon = 1e-7
        thresholds = [(i + 1) * 1.0 / (num_thresholds - 1)
                      for i in range(num_thresholds - 2)]
        thresholds = [0.0 - epsilon] + thresholds + [1.0 + epsilon]
        thresholds_tensor = tf.constant(thresholds)

        # Confusion matrix.
        values, update_ops = metrics_impl._confusion_matrix_at_thresholds(  # pylint: disable=protected-access
            labels, predictions, thresholds, weights, includes=('tp', 'fp', 'fn'))

        # Compute precision and recall at various thresholds.
        def compute_best_f1_score(tp, fp, fn, name):
            precision_at_t = math_ops.div(tp, epsilon + tp + fp,
                                          name='precision_' + name)
            recall_at_t = math_ops.div(tp, epsilon + tp + fn, name='recall_' + name)
            # Compute F1 score.
            f1_at_thresholds = (
                    2.0 * precision_at_t * recall_at_t /
                    (precision_at_t + recall_at_t + epsilon))

            best_f1 = math_ops.reduce_max(f1_at_thresholds)
            best_f1_index = tf.math.argmax(f1_at_thresholds)
            precision = precision_at_t[best_f1_index]
            recall = recall_at_t[best_f1_index]
            threshold = thresholds_tensor[best_f1_index]
            return best_f1, precision, recall, threshold

        def f1_across_replicas(_, values):
            best_f1, precision, recall, threshold = compute_best_f1_score(tp=values['tp'], fp=values['fp'],
                                                                          fn=values['fn'], name='value')
            if metrics_collections:
                ops.add_to_collections(metrics_collections, best_f1, precision, recall, threshold)
            return best_f1, precision, recall, threshold

        best_f1, precision, recall, threshold = distribution_strategy_context.get_replica_context().merge_call(
            f1_across_replicas, args=(values,))

        update_op = compute_best_f1_score(tp=update_ops['tp'], fp=update_ops['fp'],
                                          fn=update_ops['fn'], name='update')
        if updates_collections:
            ops.add_to_collections(updates_collections, update_op)

        # return (best_f1, precision, recall, threshold), update_op
        return (best_f1, update_op), (precision, update_op), (recall, update_op), (threshold, update_op)
        # return best_f1, precision, recall, threshold


def model_fn_builder(bert_config, num_labels, intent_class_weights, init_checkpoint,
                     learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        is_real_example = None
        if "is_real_example" in features:
            is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
            is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, sequence_logits, intent_logits, decode_tags, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids,
            label_ids, num_labels, intent_class_weights, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                            init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            logging_hook = tf.train.LoggingTensorHook({"loss": total_loss}, every_n_iter=10)
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                training_hooks=[logging_hook],
                scaffold_fn=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:
            def metric_fn( label_ids, decode_tags, probabilities, input_mask):
               
                best_f1, precision, recall, threshold = f1_score(label_ids, probabilities)
                evl_metrics = {}
                evl_metrics.update({'class_intent_f1': best_f1})
                evl_metrics.update({'class_precision': precision})
                evl_metrics.update({'class_recall': recall})
                evl_metrics.update({'class_threshold_': threshold})
                for i in range(num_labels):
                    best_f1, precision, recall, threshold = f1_score(label_ids[:, i], probabilities[:, i])
                    evl_metrics[f'class{i:0>2d}_f1'] = best_f1
                    evl_metrics[f'class{i:0>2d}_precision'] = precision
                    evl_metrics[f'class{i:0>2d}_recall'] = recall
                    evl_metrics[f'class{i:0>2d}_threshold'] = threshold


                for metric_name, op in evl_metrics.items():
                    tf.summary.scalar(metric_name, op[1])

                return evl_metrics

            eval_metrics = (metric_fn, [label_ids, decode_tags, probabilities, input_mask])
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metrics=eval_metrics,
                scaffold_fn=scaffold_fn)
        else:
            output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode,
                predictions={
                             "intent_predicted": probabilities,
                             "label_ids": label_ids,
                             "input_ids": input_ids,
                             "mask_length": tf.reduce_sum(input_mask, axis=1)},
                scaffold_fn=scaffold_fn)
        return output_spec

    return model_fn


# This function is not used by this file but is still used by the Colab and
# people who depend on it.
def input_fn_builder(features, seq_length, is_training, drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)
        all_label_ids.append(feature.label_ids)

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        num_examples = len(features)

        # This is for demo purposes and does NOT scale to large data sets. We do
        # not use Dataset.from_generator() because that uses tf.py_func which is
        # not TPU compatible. The right way to load data is with TFRecordReader.
        d = tf.data.Dataset.from_tensor_slices({
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "segment_ids":
                tf.constant(
                    all_segment_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "label_ids":
                tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
        })

        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        return d

    return input_fn


# This function is not used by this file but is still used by the Colab and
# people who depend on it.
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example, label_list,
                                         max_seq_length, tokenizer)

        features.append(feature)
    return features


def main():
    tf.logging.set_verbosity(tf.logging.INFO)

    processors = {
        "customized": customizedProcessor
    }

    tokenization.validate_case_matches_checkpoint( do_lower_case,
                                                   init_checkpoint)

    if not  do_train and not  do_eval and not  do_predict and not  do_export:
        raise ValueError(
            "At least one of `do_train`, `do_eval`, `do_predict' or 'do_export' must be True.")

    bert_config = modeling.BertConfig.from_json_file( bert_config_file)

    if  max_seq_length > bert_config.max_position_embeddings:
        raise ValueError(
            "Cannot use sequence length %d because the BERT model "
            "was only trained up to sequence length %d" %
            ( max_seq_length, bert_config.max_position_embeddings))

    tf.gfile.MakeDirs( output_dir)


    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    processor = processors[task_name]()

    label_map, num_labels, intent_class_weights = processor.get_labels_info()

    tokenizer = tokenization.FullTokenizer(
        vocab_file= vocab_file, do_lower_case= do_lower_case)

    tpu_cluster_resolver = None
    if use_tpu and tpu_name:
        tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
            tpu_name, zone=tpu_zone, project=gcp_project)

    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    run_config = tf.contrib.tpu.RunConfig(
        cluster=tpu_cluster_resolver,
        master=master,
        model_dir=output_dir,
        save_checkpoints_steps=save_checkpoints_steps,
        tpu_config=tf.contrib.tpu.TPUConfig(
            iterations_per_loop=iterations_per_loop,
            num_shards=num_tpu_cores,
            per_host_input_for_training=is_per_host))

    train_examples = None
    num_train_steps = None
    num_warmup_steps = None
    if do_train:
        train_examples = processor.get_train_examples(data_dir)
        num_train_steps = int(
            len(train_examples) / train_batch_size * num_train_epochs)
        num_warmup_steps = int(num_train_steps * warmup_proportion)

    model_fn = model_fn_builder(
        bert_config=bert_config,
        num_labels=num_labels,
        intent_class_weights=intent_class_weights,
        init_checkpoint=init_checkpoint,
        learning_rate=learning_rate,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=use_tpu,
        use_one_hot_embeddings=use_one_hot_embeddings)
    


    # If TPU is not available, this will fall back to normal Estimator on CPU
    # or GPU.
    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=use_tpu,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=train_batch_size,
        eval_batch_size=eval_batch_size,
        predict_batch_size=predict_batch_size)

    if do_train:
        train_file = os.path.join(output_dir, "train.tf_record")
        file_based_convert_examples_to_features(
            train_examples, label_map, max_seq_length, tokenizer, train_file)
        tf.logging.info("***** Running training *****")
        tf.logging.info("  Num examples = %d", len(train_examples))
        tf.logging.info("  Batch size = %d", train_batch_size)
        tf.logging.info("  Num steps = %d", num_train_steps)
        train_input_fn = file_based_input_fn_builder(
            input_file=train_file,
            seq_length=max_seq_length,
            is_training=True,
            drop_remainder=False,
            num_labels=num_labels)
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)


    if do_eval:
        eval_examples = processor.get_dev_examples(data_dir)
        num_actual_eval_examples = len(eval_examples)

        if use_tpu:
            # TPU requires a fixed batch size for all batches, therefore the number
            # of examples must be a multiple of the batch size, or else examples
            # will get dropped. So we pad with fake examples which are ignored
            # later on. These do NOT count towards the metric (all tf.metrics
            # support a per-instance weight, and these get a weight of 0.0).
            while len(eval_examples) % eval_batch_size != 0:
                eval_examples.append(PaddingInputExample())

        eval_file = os.path.join(output_dir, "eval.tf_record")
        file_based_convert_examples_to_features(
            eval_examples, label_map, max_seq_length, tokenizer, eval_file)

        tf.logging.info("***** Running evaluation *****")
        tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                        len(eval_examples), num_actual_eval_examples,
                        len(eval_examples) - num_actual_eval_examples)
        tf.logging.info("  Batch size = %d", eval_batch_size)

        # This tells the estimator to run through the entire set.
        eval_steps = None
        # However, if running eval on the TPU, you will need to specify the
        # number of steps.
        if use_tpu:
            assert len(eval_examples) % eval_batch_size == 0
            eval_steps = int(len(eval_examples) // eval_batch_size)

        eval_drop_remainder = True if use_tpu else False
        eval_input_fn = file_based_input_fn_builder(
            input_file=eval_file,
            seq_length=max_seq_length,
            is_training=False,
            drop_remainder=eval_drop_remainder,
            num_labels=num_labels)

        result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

        output_eval_file = os.path.join(output_dir, "eval_results.txt")
        best_threshold_file = os.path.join(output_dir, "best_class_threshold.txt")
        with tf.gfile.GFile(output_eval_file, "w") as writer:
            with tf.gfile.GFile(best_threshold_file, "w") as threshold_writer:
                tf.logging.info("***** Eval results *****")
                for key in result.keys():
                    tf.logging.info("  %s = %s", key, str(result[key]))
                    writer.write("%s = %s\n" % (key, str(result[key])))
                    if key.endswith('threshold'):
                        threshold_writer.write("%s\n" % str(result[key]))
                        tf.logging.info("")
                        writer.write("\n")

    if do_predict:
        predict_examples = processor.get_test_examples(data_dir)
        num_actual_predict_examples = len(predict_examples)
        if use_tpu:
            # TPU requires a fixed batch size for all batches, therefore the number
            # of examples must be a multiple of the batch size, or else examples
            # will get dropped. So we pad with fake examples which are ignored
            # later on.
            while len(predict_examples) % predict_batch_size != 0:
                predict_examples.append(PaddingInputExample())

        predict_file = os.path.join(output_dir, "predict.tf_record")
        file_based_convert_examples_to_features(predict_examples, label_map,
                                                max_seq_length, tokenizer, predict_file)

        tf.logging.info("***** Running prediction*****")
        tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                        len(predict_examples), num_actual_predict_examples,
                        len(predict_examples) - num_actual_predict_examples)
        tf.logging.info("  Batch size = %d", predict_batch_size)

        predict_drop_remainder = True if use_tpu else False
        predict_input_fn = file_based_input_fn_builder(
            input_file=predict_file,
            seq_length=max_seq_length,
            is_training=False,
            drop_remainder=predict_drop_remainder,
            num_labels=num_labels)

        result = estimator.predict(input_fn=predict_input_fn)
        id2label = {v: k for k, v in label_map.items()}
        output_predict_file = os.path.join(output_dir, "test_results.tsv")

        best_threshold_file = os.path.join(output_dir, "best_class_threshold.txt")
        if not tf.gfile.Exists(best_threshold_file):
            threshold = [0.5] * num_labels
        else:
            with tf.gfile.GFile(best_threshold_file, "r") as reader:
                threshold = reader.read().splitlines()
            threshold = list(map(float, threshold))

        with tf.gfile.GFile(output_predict_file, "w") as writer:
            num_written_lines = 0
            output_line = "\t".join(
                ['frame_O_X', 'intent OX', 'intent_label', 'intent_pred', 'sentence']) + "\n"
            writer.write(output_line)

            # below for cal metrics
            slot_pred_all_data = []
            slot_label_all_data = []
            intent_pred_all_data = []
            intent_label_all_data = []

            for index, item in enumerate(result):
                mask_length = item["mask_length"]
                intent_predicted = item["intent_predicted"]
                intent_predicted = [intent_predicted[i] > threshold[i] for i in range(num_labels)]

                label_ids = item["label_ids"]
                input_ids = item["input_ids"][:mask_length]
                tokens = tokenizer.convert_ids_to_tokens(input_ids)

                intent_pred_all_data.append(intent_predicted)
                intent_label_all_data.append(label_ids)

                pre_intent = ' '.join([id2label[index] for index, pred in enumerate(intent_predicted) if pred])
                label_intent = ' '.join([id2label[index] for index, label in enumerate(label_ids) if label])

                sentence = ''.join(tokens)
                intent_compare = 'O' if pre_intent == label_intent else 'X'
                frame_O_X = 'O' if pre_intent == label_intent else 'X'
                output_line = "\t".join(
                    [str(item["intent_predicted"]), label_intent, pre_intent, sentence]) + "\n"
                writer.write(output_line)
                num_written_lines += 1
            assert num_written_lines == num_actual_predict_examples

        # write test metrics
        from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
        import numpy as np
        output_test_metrics_file = os.path.join(output_dir, "test_results.txt")
        with tf.gfile.GFile(output_test_metrics_file, "w") as writer:

            intent_label_all_data = np.array(intent_label_all_data)
            intent_pred_all_data = np.array(intent_pred_all_data)

            for i in range(num_labels):
                f1 = f1_score(intent_label_all_data[:, i], intent_pred_all_data[:, i])
                precision = precision_score(intent_label_all_data[:, i], intent_pred_all_data[:, i])
                recall = recall_score(intent_label_all_data[:, i], intent_pred_all_data[:, i])

                writer.write(f'class{i:0>2d}_f1 = {f1}\n')
                writer.write(f'class{i:0>2d}_precision = {precision}\n')
                writer.write(f'class{i:0>2d}_recall = {recall}\n')
                writer.write('\n\n')

            intent_label_flatten = intent_label_all_data.reshape(-1)
            intent_pred_flatten = intent_pred_all_data.reshape(-1)
            intent_f1 = f1_score(intent_label_flatten, intent_pred_flatten)
            intent_precision = precision_score(intent_label_flatten, intent_pred_flatten)
            intent_recall = recall_score(intent_label_flatten, intent_pred_flatten)

            writer.write(f'class_intent_f1 = {intent_f1}\n')
            writer.write(f'class_intent_precision = {intent_precision}\n')
            writer.write(f'class_intent_recall = {intent_recall}\n')
            writer.write('\n\n')

    if do_export:
        def serving_input_fn():
            label_ids = tf.placeholder(tf.int32, [None, num_labels], name='label_ids')
            input_ids = tf.placeholder(tf.int32, [None, max_seq_length], name='input_ids')
            input_mask = tf.placeholder(tf.int32, [None, max_seq_length], name='input_mask')
            segment_ids = tf.placeholder(tf.int32, [None, max_seq_length], name='segment_ids')
            input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
                'label_ids': label_ids,
                'input_ids': input_ids,
                'input_mask': input_mask,
                'segment_ids': segment_ids
            })()
            return input_fn

        estimator._export_to_tpu = False
        estimator.export_saved_model(export_dir, serving_input_fn)


if __name__ == "__main__":
    main()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=['事件_感情', '事件_成長', '事件_金錢', '感受內在感受_創傷小孩', '感受內在感受_孤單小孩', '感受內在感受_憤怒小孩', '認知想法_差評', '認知想法_控制欲'], y=['感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_孤單小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感受_憤怒小孩', '感受內在感